# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.
%load_ext dotenv
%dotenv

In [2]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob

# Write your code below.
PRICE_DATA = os.getenv('PRICE_DATA')

# Create a list of paths to all parquet files within the PRICE_DATA directory
parquet_files = glob(os.path.join(PRICE_DATA, '**/*.parquet'), recursive=True)

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [4]:
# Write your code below.

# Read the parquet files as a dask dataframe
dd_px = dd.read_parquet(parquet_files).set_index("ticker")
#print(dd_px.compute().head())

# Add lags for variables Close and Adj. Close using apply function after grouping by ticker.
# Calculate returns and hi_lo_range using lambda functions within assign.
dd_feat = dd_px.groupby('ticker',group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1),
                       Adj_Close_lag_1 = x['Adj Close'].shift(1))
).assign(
    returns = lambda x: x['Close']/x["Close_lag_1"] - 1,
    hi_lo_range = lambda x: x['High'] - x['Low']
)
dd_feat

/var/folders/s7/1h5812v56198slm5x05gnjl00000gn/T/ipykernel_92232/1867136161.py:9: UserWarning: `meta` is not specified, inferred from partial data.
Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result

  dd_feat = dd_px.groupby('ticker',group_keys=False).apply(


,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range
npartitions=89,,,,,,,,,,,,,
A,datetime64[ns],float64,float64,float64,float64,float64,float64,string,int32,float64,float64,float64,float64
ACB,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZEUS,...,...,...,...,...,...,...,...,...,...,...,...,...
ZEUS,...,...,...,...,...,...,...,...,...,...,...,...,...


In [9]:
dd_feat.compute()

,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range
ticker,,,,,,,,,,,,,
A,1999-11-18,32.546494,35.765381,28.612303,31.473534,27.068665,62546300.0,A.csv,1999,NaN,NaN,NaN,7.153078
A,1999-11-19,30.713520,30.758226,28.478184,28.880543,24.838577,15234100.0,A.csv,1999,31.473534,27.068665,-0.082386,2.280043
A,1999-11-22,29.551144,31.473534,28.657009,31.473534,27.068665,6577800.0,A.csv,1999,28.880543,24.838577,0.089783,2.816525
A,1999-11-23,30.400572,31.205294,28.612303,28.612303,24.607880,5975600.0,A.csv,1999,31.473534,27.068665,-0.090909,2.592991
A,1999-11-24,28.701717,29.998211,28.612303,29.372318,25.261524,4843200.0,A.csv,1999,28.612303,24.607880,0.026563,1.385908
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZEUS,2020-03-26,9.610000,9.940000,9.260000,9.590000,9.590000,60500.0,ZEUS.csv,2020,9.670000,9.670000,-0.008273,0.679999
ZEUS,2020-03-27,9.330000,9.330000,8.700000,8.700000,8.700000,52900.0,ZEUS.csv,2020,9.590000,9.590000,-0.092805,0.630000
ZEUS,2020-03-30,8.810000,9.760000,8.700000,9.680000,9.680000,73700.0,ZEUS.csv,2020,8.700000,8.700000,0.112644,1.060000


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [ ]:
# Write your code below.

# Convert the Dask data frame to a pandas dataframe
df = dd_feat.compute().reset_index()
df                                         


,ticker,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range
0,A,1999-11-18,32.546494,35.765381,28.612303,31.473534,27.068665,62546300.0,A.csv,1999,NaN,NaN,NaN,7.153078
1,A,1999-11-19,30.713520,30.758226,28.478184,28.880543,24.838577,15234100.0,A.csv,1999,31.473534,27.068665,-0.082386,2.280043
2,A,1999-11-22,29.551144,31.473534,28.657009,31.473534,27.068665,6577800.0,A.csv,1999,28.880543,24.838577,0.089783,2.816525
3,A,1999-11-23,30.400572,31.205294,28.612303,28.612303,24.607880,5975600.0,A.csv,1999,31.473534,27.068665,-0.090909,2.592991
4,A,1999-11-24,28.701717,29.998211,28.612303,29.372318,25.261524,4843200.0,A.csv,1999,28.612303,24.607880,0.026563,1.385908
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324306,ZEUS,2004-12-27,28.000000,29.230000,27.690001,28.670000,26.258089,651600.0,ZEUS.csv,2004,27.670000,25.342215,0.036140,1.539999
324307,ZEUS,2004-12-28,28.900000,30.200001,28.700001,29.900000,27.384602,529400.0,ZEUS.csv,2004,28.670000,26.258089,0.042902,1.500000
324308,ZEUS,2004-12-29,30.280001,30.299999,29.030001,29.070000,26.624434,355300.0,ZEUS.csv,2004,29.900000,27.384602,-0.027759,1.269999
324309,ZEUS,2004-12-30,28.450001,28.450001,25.330000,26.170000,23.968391,1361800.0,ZEUS.csv,2004,29.070000,26.624434,-0.099759,3.120001


In [36]:
df_result = df.groupby("ticker", group_keys=False).apply(
    lambda x: x.assign(moving_10_day_return_average = x['returns'].rolling(10).mean()))

# Check the results for any one ticker
df_result[df_result['ticker']=='ZEUS']

# The first 9 rows of the new column will be NaN because there aren't 10 preceding entries to calculate the rolling mean. 
# The columns starts to get populated once there are 10 preceding entries.
df_result

/var/folders/s7/1h5812v56198slm5x05gnjl00000gn/T/ipykernel_79718/3340076701.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_result = df.groupby("ticker", group_keys=False).apply(


,ticker,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range,moving_10_day_return_average
0,A,1999-11-18,32.546494,35.765381,28.612303,31.473534,27.068665,62546300.0,A.csv,1999,NaN,NaN,NaN,7.153078,NaN
1,A,1999-11-19,30.713520,30.758226,28.478184,28.880543,24.838577,15234100.0,A.csv,1999,31.473534,27.068665,-0.082386,2.280043,NaN
2,A,1999-11-22,29.551144,31.473534,28.657009,31.473534,27.068665,6577800.0,A.csv,1999,28.880543,24.838577,0.089783,2.816525,NaN
3,A,1999-11-23,30.400572,31.205294,28.612303,28.612303,24.607880,5975600.0,A.csv,1999,31.473534,27.068665,-0.090909,2.592991,NaN
4,A,1999-11-24,28.701717,29.998211,28.612303,29.372318,25.261524,4843200.0,A.csv,1999,28.612303,24.607880,0.026563,1.385908,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324306,ZEUS,2004-12-27,28.000000,29.230000,27.690001,28.670000,26.258089,651600.0,ZEUS.csv,2004,27.670000,25.342215,0.036140,1.539999,0.020963
324307,ZEUS,2004-12-28,28.900000,30.200001,28.700001,29.900000,27.384602,529400.0,ZEUS.csv,2004,28.670000,26.258089,0.042902,1.500000,0.023978
324308,ZEUS,2004-12-29,30.280001,30.299999,29.030001,29.070000,26.624434,355300.0,ZEUS.csv,2004,29.900000,27.384602,-0.027759,1.269999,0.025104
324309,ZEUS,2004-12-30,28.450001,28.450001,25.330000,26.170000,23.968391,1361800.0,ZEUS.csv,2004,29.070000,26.624434,-0.099759,3.120001,0.004783


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

- It is not necessary to convert to pandas to calculate the moving average return. The same operation can be performed in dask. However, with default settings below, I get an error about number of partitions, "NotImplementedError: Partition size is less than overlapping window size. Try using ``df.repartition`` to increase the partition size." I am not entirely certain how to deal with this error. I decreased the number of partitions, but then the order of rows in the resulting dataframe seems to have changed. For example, the first entry in dd_result below for ticker A is now from 2014-07-03, whereas after pandas calculations, the first entry for ticker A was 1999-11-18

In [17]:
dd_feat_repartitioned = dd_feat.repartition(npartitions=10)
dd_result = dd_feat_repartitioned.assign(
    moving_10_day_return_average = lambda x: x['returns'].rolling(10).mean()
)
dd_result.compute().reset_index()

,ticker,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range,moving_10_day_return_average
0,A,2014-07-03,41.623749,41.874107,41.502148,41.816883,39.522949,839700.0,A.csv,2014,NaN,NaN,NaN,0.371960,NaN
1,A,2014-07-07,41.630901,41.666668,41.380543,41.573677,39.293072,1326900.0,A.csv,2014,41.816883,39.522949,-0.005816,0.286125,NaN
2,A,2014-07-08,41.373390,41.437767,40.715309,40.879829,38.637295,2259000.0,A.csv,2014,41.573677,39.293072,-0.016690,0.722458,NaN
3,A,2014-07-09,40.808296,40.829758,40.429184,40.701000,38.468281,1787300.0,A.csv,2014,40.879829,38.637295,-0.004375,0.400574,NaN
4,A,2014-07-10,40.350502,40.507870,40.064377,40.407726,38.191090,1662600.0,A.csv,2014,40.701000,38.468281,-0.007206,0.443493,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324306,ZEUS,2004-12-27,28.000000,29.230000,27.690001,28.670000,26.258089,651600.0,ZEUS.csv,2004,27.670000,25.342215,0.036140,1.539999,0.020963
324307,ZEUS,2004-12-28,28.900000,30.200001,28.700001,29.900000,27.384602,529400.0,ZEUS.csv,2004,28.670000,26.258089,0.042902,1.500000,0.023978
324308,ZEUS,2004-12-29,30.280001,30.299999,29.030001,29.070000,26.624434,355300.0,ZEUS.csv,2004,29.900000,27.384602,-0.027759,1.269999,0.025104
324309,ZEUS,2004-12-30,28.450001,28.450001,25.330000,26.170000,23.968391,1361800.0,ZEUS.csv,2004,29.070000,26.624434,-0.099759,3.120001,0.004783


In [18]:
test_df = dd_result.compute().reset_index()
test_df

,ticker,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range,moving_10_day_return_average
0,A,1999-11-18,32.546494,35.765381,28.612303,31.473534,27.068665,62546300.0,A.csv,1999,NaN,NaN,NaN,7.153078,NaN
1,A,1999-11-19,30.713520,30.758226,28.478184,28.880543,24.838577,15234100.0,A.csv,1999,31.473534,27.068665,-0.082386,2.280043,NaN
2,A,1999-11-22,29.551144,31.473534,28.657009,31.473534,27.068665,6577800.0,A.csv,1999,28.880543,24.838577,0.089783,2.816525,NaN
3,A,1999-11-23,30.400572,31.205294,28.612303,28.612303,24.607880,5975600.0,A.csv,1999,31.473534,27.068665,-0.090909,2.592991,NaN
4,A,1999-11-24,28.701717,29.998211,28.612303,29.372318,25.261524,4843200.0,A.csv,1999,28.612303,24.607880,0.026563,1.385908,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324306,ZEUS,2004-12-27,28.000000,29.230000,27.690001,28.670000,26.258089,651600.0,ZEUS.csv,2004,27.670000,25.342215,0.036140,1.539999,0.020963
324307,ZEUS,2004-12-28,28.900000,30.200001,28.700001,29.900000,27.384602,529400.0,ZEUS.csv,2004,28.670000,26.258089,0.042902,1.500000,0.023978
324308,ZEUS,2004-12-29,30.280001,30.299999,29.030001,29.070000,26.624434,355300.0,ZEUS.csv,2004,29.900000,27.384602,-0.027759,1.269999,0.025104
324309,ZEUS,2004-12-30,28.450001,28.450001,25.330000,26.170000,23.968391,1361800.0,ZEUS.csv,2004,29.070000,26.624434,-0.099759,3.120001,0.004783


## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.